# TEMPLATE PARA SUBIR ARCHIVOS A LA NUBE

# IMPORT GLOBALES

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import sys

## LECTURA DE ARCHIVOS

In [2]:
csv_file = "tu.csv"
df = pd.read_csv(csv_file)
print("Total number of rows = %s" % len(df.index))
df.head(3)

Total number of rows = 3004


,business_id,name,address,post_code,city,state,latitude,longitude,categories,attributes,avg_rating,num_of_reviews,url,source
0,uczmbBk5O3tYhGue13dCDg,New Orleans Spirit Tours,723 St Peter St,NaN,New Orleans,IN,29.958431,-90.065173,"['Hotels & Travel', ' Tours', ' Local Flavor']","{'WiFi': ""u'no'""}",4.0,NaN,NaN,NaN
1,Nh1rc9aSeO-Y5lYmXVS6CA,Quality Inn Nashville Downtown - Stadium,303 Interstate Dr,NaN,Nashville,AZ,36.168834,-86.766911,"['Hotels & Travel', ' Event Planning & Service...","{'WiFi': ""'free'"", 'BusinessAcceptsCreditCards...",2.0,NaN,NaN,NaN
2,rh3C_JK4NAoUJ-HNStvHtw,"Historic Philadelphia, Inc","600 Walnut St, Ste 130",NaN,Philadelphia,AZ,39.948321,-75.151952,"['Hotels & Travel', ' Tours']",NaN,4.5,NaN,NaN,NaN


In [3]:
df.columns

Index(['business_id', 'name', 'address', 'post_code', 'city', 'state',
       'latitude', 'longitude', 'categories', 'attributes', 'avg_rating',
       'num_of_reviews', 'url', 'source'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   business_id     3004 non-null   object 
 1   name            3004 non-null   object 
 2   address         2921 non-null   object 
 3   post_code       0 non-null      float64
 4   city            3004 non-null   object 
 5   state           3004 non-null   object 
 6   latitude        3004 non-null   float64
 7   longitude       3004 non-null   float64
 8   categories      3004 non-null   object 
 9   attributes      2373 non-null   object 
 10  avg_rating      3004 non-null   float64
 11  num_of_reviews  0 non-null      float64
 12  url             0 non-null      float64
 13  source          0 non-null      float64
dtypes: float64(7), object(7)
memory usage: 328.7+ KB


## FUNCIONES

In [6]:
param_dic = {
    "host"      : "yelp-proyect-instancia.csp6hdqhq32b.us-east-2.rds.amazonaws.com",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "9J1aLQKrVNX58ejADdXu"
}

In [7]:

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [8]:

def execute_query(conn, query):
    """  Esta función ejecuta consultas """
    
    ret = 0 # Valor de retorno
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # Si fue una consulta SELECT, devolver el resultado
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret

In [23]:
def create_insert_query(row):
    return f"""
        INSERT INTO yelp_hotel_business 
        VALUES (
            '{row['NAME_1']}',
            '{row['NAME_2']}',
            '{row['NAME_3']}'
            );
        """

In [25]:
try:
    #Realizo la conexion 
    conn = connect(param_dic)

    # Crear la tabla en la base de datos
    create_table_query = """
    DROP TABLE IF EXISTS yelp_hotel_business;
    
    CREATE TABLE yelp_hotel_business (
    NAME_1 varchar(500),
    NAME_2 varchar(500), 
    NAME_3 float
    );

    """

    # Ejecutar la consulta para crear la tabla
    execute_query(conn, create_table_query)

    # Insertar datos desde el DataFrame a la tabla
    for index, row in df.iterrows():
        insert_query = create_insert_query(row)
        execute_query(conn, insert_query)

except (Exception, psycopg2.DatabaseError) as error:
    print("Error durante la ejecución: %s" % error)

finally:
    # Cerrar la conexión a la base de datos, independientemente de si hay una excepción o no
    if conn is not None:
        conn.close()

Connecting to the PostgreSQL database...
Connection successful
